### Using the approach that worked in feature selection 2, here I will compare different approaches <br>
specifcially try to see what features were important, the influence of the doubling, stratified splitting etc

In [2]:
import numpy as np
import pandas as pd
import _pickle as pickle

import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [4]:
with open('./pickles/allFeats2XrecordsFrame.pkl', 'rb') as inp:
    dataDouble = pickle.load(inp)
print(f"Data has %s records and %s columns" % (len(dataDouble),int(dataDouble.size/len(dataDouble))))

with open('./pickles/allFeats1XrecordsFrame.pkl', 'rb') as inp:
    dataSingle = pickle.load(inp)
print(f"Data has %s records and %s columns" % (len(dataSingle),int(dataSingle.size/len(dataSingle))))
dataSingle.head(2)

Data has 533852 records and 147 columns
Data has 266926 records and 147 columns


,Drug1,Drug2,Cell line,ZIP,Bliss,Loewe,HSA,D1 cIds,D1 Eccentricity,D1 InertialShapeFactor,...,D2 SMR_VSA3,D2 SlogP_VSA1,D2 SlogP_VSA2,D2 SlogP_VSA3,D2 EState_VSA1,D2 EState_VSA2,D2 EState_VSA3,D2 VSA_EState1,D2 VSA_EState2,D2 VSA_EState3
192,bez-235,mk-8776,A2058,9.73,8.15,10.19,15.11,CIDs11977753,0.894284,0.0002,...,29.695332,11.050456,37.468056,7.047672,0.0,0.0,11.735769,4.304319,4.901404,12.064533
192,bez-235,mk-8776,A2058,9.73,8.15,10.19,15.11,CIDs11977753,0.894284,0.0002,...,29.695332,11.050456,37.468056,7.047672,0.0,0.0,11.735769,4.304319,4.901404,12.064533


Compute Decision Tree an mean absolute error for both single & double data

In [3]:
Y = dataDouble.iloc[:,6]
X = dataDouble.iloc[:,8:]
X.reset_index(inplace=True)
X.drop(["D1 Ipc","D2 Ipc","D2 cIds","index"],axis=1, inplace=True) # Ipc has too extreme values; no need for cIds and index

X_train,x_test,Y_train,y_test = train_test_split(X,Y,test_size=0.3)

#decision tree training is O(N)
regressor = DecisionTreeRegressor()
regressor.fit(X_train,Y_train)

predsTrain=regressor.predict(X_train)#.iloc[100:105,:]
predsTest=regressor.predict(x_test)

print(np.mean(abs(predsTest-y_test)))
print(np.mean(abs(predsTrain-Y_train)))

5.550246120049201
3.1907523333566568


In [3]:
Ysingle = dataSingle.iloc[:,6]
Xsingle = dataSingle.iloc[:,8:]
Xsingle.reset_index(inplace=True)
Xsingle.drop(["D1 Ipc","D2 Ipc","D2 cIds","index"],axis=1, inplace=True) # Ipc has too extreme values; no need for cIds and index

X_trainS,x_testS,Y_trainS,y_testS = train_test_split(Xsingle,Ysingle,test_size=0.3)

#decision tree training is O(N)
regressorS = DecisionTreeRegressor()
regressorS.fit(X_trainS,Y_trainS)

predsTrainS=regressorS.predict(X_trainS)#.iloc[100:105,:]
predsTestS=regressorS.predict(x_testS)

print(np.mean(abs(predsTestS-y_testS)))
print(np.mean(abs(predsTrainS-Y_trainS)))

5.371067685993316
3.1668374075469226


This proves that the features found are not due to patterns detected in the double-equivalent of the data<br>
<br>
I will no see if removing low variance features and highly correlated ones reduce the accuracy
<br> or just figure important features with RF and then drop everything else?

# check "delete" file

In [1]:
import numpy as np
import pandas as pd
import _pickle as pickle
with open('./pickles/doubledData.pkl', 'rb') as inp:
    dataDouble = pickle.load(inp)

with open('./pickles/drugMACCS.pkl', 'rb') as inp:
    drugs = pickle.load(inp)

drugs["drugName"]=drugs["drugName"].str.lower()
drugs.set_index("drugName",inplace=True)

# removing some features because Kernel crashes otherwise
dropLst = []
for i in range(len(drugs.iloc[0])):
    if drugs.iloc[:,i].mean()<0.22 or drugs.iloc[:,i].mean()>1:
        dropLst.append(i)
drugs.drop(columns=dropLst, inplace=True)

d1_drugs = drugs.add_prefix("D1 ")
d2_drugs = drugs.add_prefix("D2 ")


dataMACCS2 = dataDouble.join(d1_drugs,on="Drug1",how="left")

dataMACCS2 = dataMACCS2.join(d2_drugs,on="Drug2",how="left")
dataMACCS2.dropna(inplace=True)
dataMACCS2

#with open('dataMACCS2.pkl', 'wb') as outp:
#    pickle.dump(dataMACCS2, outp)

,Drug1,Drug2,Cell line,ZIP,Bliss,Loewe,HSA,D1 38,D1 42,D1 57,...,D2 156,D2 157,D2 158,D2 159,D2 160,D2 161,D2 162,D2 163,D2 164,D2 165
0,5-fu,abt-888,A2058,1.720,6.260,-2.750,5.540,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,5-fu,abt-888,A2058,5.880,12.330,3.330,11.610,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,5-fu,abt-888,A2058,3.590,11.660,2.650,10.940,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,5-fu,abt-888,A2058,-0.850,5.150,-3.860,4.430,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,5-fu,azd1775,A2058,12.290,15.770,10.400,18.660,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997682,mebendazole,orlistat,DIPG25,-6.509,-2.023,-11.290,-5.623,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0
997683,mebendazole,rotenone,DIPG25,-0.578,-0.210,1.802,4.119,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
997684,mebendazole,cyanein,DIPG25,6.855,3.698,-3.700,3.262,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0
997685,mebendazole,erlotinib,DIPG25,-1.653,6.694,2.185,3.710,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [3]:
Y = dataMACCS2.iloc[:,6]
X = dataMACCS2.iloc[:,7:]
X.reset_index(inplace=True)
X.drop(["index"],axis=1, inplace=True)

X_train,x_test,Y_train,y_test = train_test_split(X,Y,test_size=0.3,stratify=Y)

#decision tree training is O(N)
regressor = DecisionTreeRegressor()
regressor.fit(X_train,Y_train)

predsTrain=regressor.predict(X_train)#.iloc[100:105,:]
predsTest=regressor.predict(x_test)

print(np.mean(abs(predsTest-y_test)))
print(np.mean(abs(predsTrain-Y_train)))

5.491077800629799
3.4072603129446755


In [10]:
pd.DataFrame([regressor.feature_names_in_,regressor.feature_importances_]).transpose().sort_values(by=1)

,0,1
175,D2 165,0.000256
87,D1 165,0.000307
171,D2 161,0.000319
153,D2 143,0.000359
44,D1 121,0.000379
...,...,...
22,D1 94,0.012079
173,D2 163,0.069233
137,D2 126,0.073723
83,D1 161,0.197634


Same issue, that is not the Kind of pattern I am looking for